<a href="https://colab.research.google.com/github/jiiiisoo/ICT/blob/main/helmet_detection/helmet_detection_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential,models,layers
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dropout,Flatten,Dense,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import xml.etree.ElementTree as et
from sklearn.model_selection import train_test_split
import random

import cv2
import imutils
import matplotlib.pyplot as plt

In [ ]:
#unzip image data file

!unzip -uq "/content/drive/MyDrive/deeplearning/final_project/archive.zip" -d "/content/drive/MyDrive/deeplearning/final_project"

In [ ]:
X=[]
Y=[]

#image 불러오기
image_list = os.listdir("/content/drive/MyDrive/deeplearning/final_project/images")
label_list = os.listdir("/content/drive/MyDrive/deeplearning/final_project/annotations")

print(len(image_list), len(label_list))

764 764


In [ ]:
#label: With Helmet=1, Without Helmet=0
width=128
height=128

image_set=[]
for img in image_list:
    label=[]
    classnum = []
    xtree = et.parse('/content/drive/MyDrive/deeplearning/final_project/annotations/' + img.split('.')[0] + '.xml')
    #size = xtree.find('size')
    cv2_image = cv2.imread(os.path.join('/content/drive/MyDrive/deeplearning/final_project/images', img),cv2.IMREAD_COLOR)
    resized = cv2.resize(cv2_image, (width,height)) #보통 이미지 축소에는 INTER_AREA 사용

    for e in xtree.findall('object'):
      name = e.find('name').text
      label.append({'name': name,})
      if name == 'Without Helmet':
        label_num=0
      elif name == 'With Helmet':
        label_num=1
      #name이 둘 다 아닌 경우가 있을 수 있으므로
      else:
        label_num=2
    image_set.append((resized, label_num))
    
print(len(image_set))

764


In [ ]:
#Split data into train, test dataset

random.shuffle(image_set)

train_set, test_set = train_test_split(image_set, test_size=0.2, random_state=1)

print(len(train_set), len(test_set), len(train_label))

611 153 611


In [ ]:
# image, label 분리
train_image=[]
train_label=[]
test_image=[]
test_label=[]

for image, label in train_set:
  train_image.append(image)
  train_label.append(label)

for image, label in test_set:
  test_image.append(image)
  test_label.append(label)

print(len(train_image))
#차원 조정 ->>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>왱?!
train_image=np.array(train_image)#.reshape(-1,width,height,1)
print(train_image.shape)
test_image=np.array(test_image)
print(train_image.shape)

train_label=np.array(train_label).reshape((-1,1))
test_label=np.array(test_label).reshape((-1,1))

# Normalize pixel values to be between 0 and 1
train_image=train_image/255.0

#image 증강 통해 overfitting 방지하고자 imagedatagenerator로 이미지 변형



611
(611, 128, 128, 3)
(611, 128, 128, 3)


In [ ]:

datagen = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    zoom_range=0.4,
    horizontal_flip=True)

In [ ]:
len(train_image)

611

In [ ]:
train_datagen = datagen.flow(train_image, train_label, batch_size=64)
test_datagen = datagen.flow(test_image, test_label, batch_size=64)

In [ ]:
# model 구축

model=models.Sequential()
model.add(layers.Conv2D(64,(3,3),activation='relu',input_shape=(128,128,3)))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPool2D(2,2))
#model.add(layers.Dropout(0.2))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPool2D(2,2))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(1,activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 126, 126, 64)      1792      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 63, 63, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 61, 61, 32)        18464     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 14, 14, 32)      

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=train_image.shape[1:], data_format='channels_last',))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
# model fit
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer = 'rmsprop', metrics=['accuracy'])

# Fitting the neural network to the training set
history = model.fit(train_datagen, epochs=10, validation_data=(test_datagen))

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


10/10 [==============================] - 7s 488ms/step - loss: 3.0084 - accuracy: 0.5663 - val_loss: 0.6739 - val_accuracy: 0.6275
Epoch 2/10
10/10 [==============================] - 3s 279ms/step - loss: 1.4896 - accuracy: 0.5745 - val_loss: 0.6738 - val_accuracy: 0.6209
Epoch 3/10
10/10 [==============================] - 3s 287ms/step - loss: 1.1327 - accuracy: 0.5859 - val_loss: 0.7181 - val_accuracy: 0.6209
Epoch 4/10
10/10 [==============================] - 3s 288ms/step - loss: 1.0431 - accuracy: 0.5581 - val_loss: 0.7830 - val_accuracy: 0.6209
Epoch 5/10
10/10 [==============================] - 3s 299ms/step - loss: 0.8451 - accuracy: 0.6187 - val_loss: 0.8451 - val_accuracy: 0.6209
Epoch 6/10
10/10 [==============================] - 3s 294ms/step - loss: 0.8675 - accuracy: 0.5925 - val_loss: 0.7420 - val_accuracy: 0.6209
Epoch 7/10
10/10 [==============================] - 3s 294ms/step - loss: 0.8082 - accuracy: 0.5859 - val_loss: 0.7026 - val_accuracy: 0.6209
Epoch 8/10
10/10 

reference  
https://hwiyong.tistory.com/48  
https://keras.io/ko/preprocessing/image/  
https://yunwoong.tistory.com/127